In [ ]:
# STEP 1: Install Kaggle API and import necessary libraries
!pip install -q kaggle
import os
import zipfile

# STEP 2: Create the Kaggle directory and upload kaggle.json (already done by you)
kaggle_path = "/root/.kaggle"
os.makedirs(kaggle_path, exist_ok=True)
# If not uploaded, you'd use:
# from google.colab import files
# files.upload()  # upload kaggle.json manually here

# STEP 3: Move kaggle.json to the correct directory
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

# STEP 4: Download the dataset from Kaggle
!kaggle datasets download -d debeshjha1/kvasirseg

# STEP 5: Unzip the dataset
with zipfile.ZipFile("kvasirseg.zip", 'r') as zip_ref:
    zip_ref.extractall("kvasirseg")

# STEP 6: Verify the extracted contents
import os
base_path = "kvasirseg/Kvasir-SEG"
for root, dirs, files in os.walk(base_path):
    print(f"{root}: {len(files)} files")


Dataset URL: https://www.kaggle.com/datasets/debeshjha1/kvasirseg
License(s): copyright-authors
  0% 0.00/144M [00:00<?, ?B/s]
100% 144M/144M [00:00<00:00, 1.77GB/s]
kvasirseg/Kvasir-SEG: 0 files
kvasirseg/Kvasir-SEG/Kvasir-SEG: 1 files
kvasirseg/Kvasir-SEG/Kvasir-SEG/images: 1000 files
kvasirseg/Kvasir-SEG/Kvasir-SEG/annotated_images: 1000 files
kvasirseg/Kvasir-SEG/Kvasir-SEG/masks: 1000 files
kvasirseg/Kvasir-SEG/Kvasir-SEG/bbox: 1000 files


In [ ]:
import os

def print_limited_structure(folder_path, file_limit=2):
    for root, dirs, files in os.walk(folder_path):
        level = root.replace(folder_path, '').count(os.sep)
        indent = '    ' * level
        print(f"{indent}{os.path.basename(root)}/")

        sub_indent = '    ' * (level + 1)
        for i, f in enumerate(files):
            if i < file_limit:
                print(f"{sub_indent}{f}")
            elif i == file_limit:
                print(f"{sub_indent}... + {len(files) - file_limit} more files")
                break

# Example usage
folder_path = "kvasirseg/Kvasir-SEG"  # Change to your folder path
print_limited_structure(folder_path)


Kvasir-SEG/
    Kvasir-SEG/
        1911.07069.pdf
        images/
            cju8914beokbf0850isxpocrk.jpg
            cju424hy5lckr085073fva1ok.jpg
            ... + 998 more files
        annotated_images/
            cju8914beokbf0850isxpocrk.jpg
            cju424hy5lckr085073fva1ok.jpg
            ... + 998 more files
        masks/
            cju8914beokbf0850isxpocrk.jpg
            cju424hy5lckr085073fva1ok.jpg
            ... + 998 more files
        bbox/
            cju2zdvjn9h7r08553cp4eed5.csv
            cju32phw2bv130801yj7bkouq.csv
            ... + 998 more files


In [ ]:
import pandas as pd
import os

bbox_dir = "/content/kvasirseg/Kvasir-SEG/Kvasir-SEG/bbox"
image_dir = "/content/kvasirseg/Kvasir-SEG/Kvasir-SEG/images"

# Read one sample
csv_file = os.path.join(bbox_dir, "cju843yjskhq30818qre4rwm2.csv")
df = pd.read_csv(csv_file)
print(df.head())


  class_name  xmin  ymin  xmax  ymax
0      polyp   334    84   429   299


In [ ]:
import os
import pandas as pd
from PIL import Image
import shutil
import random

# Set paths
base_dir = "/content/kvasirseg"
images_dir = os.path.join(base_dir, "/content/kvasirseg/Kvasir-SEG/Kvasir-SEG/images")
bbox_dir = os.path.join(base_dir, "/content/kvasirseg/Kvasir-SEG/Kvasir-SEG/bbox")

# Output folders
output_dir = "kvasir_yolo"
images_output = os.path.join(output_dir, "images")
labels_output = os.path.join(output_dir, "labels")

for folder in ["train", "val"]:
    os.makedirs(os.path.join(images_output, folder), exist_ok=True)
    os.makedirs(os.path.join(labels_output, folder), exist_ok=True)

# Helper to convert to YOLO format
def convert_bbox(xmin, ymin, xmax, ymax, img_w, img_h):
    x_center = (xmin + xmax) / 2 / img_w
    y_center = (ymin + ymax) / 2 / img_h
    width = (xmax - xmin) / img_w
    height = (ymax - ymin) / img_h
    return [x_center, y_center, width, height]

# Prepare dataset list
all_files = [f for f in os.listdir(bbox_dir) if f.endswith(".csv")]
random.shuffle(all_files)
split_idx = int(len(all_files) * 0.8)
train_files = all_files[:split_idx]
val_files = all_files[split_idx:]

def process_files(file_list, split):
    for csv_file in file_list:
        image_id = csv_file.replace(".csv", "")
        image_path = os.path.join(images_dir, image_id + ".jpg")
        label_path = os.path.join(labels_output, split, image_id + ".txt")

        if not os.path.exists(image_path):
            continue

        # Load image to get size
        with Image.open(image_path) as img:
            w, h = img.size

        # Read bbox CSV
        df = pd.read_csv(os.path.join(bbox_dir, csv_file))

        # Write YOLO format
        with open(label_path, "w") as f:
            for _, row in df.iterrows():
                if row['class_name'].lower() != 'polyp':
                    continue
                xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']
                x_center, y_center, width, height = convert_bbox(xmin, ymin, xmax, ymax, w, h)
                f.write(f"0 {x_center} {y_center} {width} {height}\n")

        # Copy image
        shutil.copy(image_path, os.path.join(images_output, split, image_id + ".jpg"))

# Process all files
process_files(train_files, "train")
process_files(val_files, "val")

# Create data.yaml
yaml_text = f"""
path: {os.path.abspath(output_dir)}
train: images/train
val: images/val

nc: 1
names: ['polyp']
"""

with open(os.path.join(output_dir, "data.yaml"), "w") as f:
    f.write(yaml_text.strip())

print("✅ YOLO dataset prepared successfully!")



✅ YOLO dataset prepared successfully!


In [ ]:
import os
import pandas as pd
from PIL import Image
import shutil
import random

# Set paths
base_dir = "/content/kvasirseg/Kvasir-SEG/Kvasir-SEG"
images_dir = os.path.join(base_dir, "images")
bbox_dir = os.path.join(base_dir, "bbox")

# Output folders
output_dir = "kvasir_yolo"
images_output = os.path.join(output_dir, "images")
labels_output = os.path.join(output_dir, "labels")

for folder in ["train", "val", "test"]:
    os.makedirs(os.path.join(images_output, folder), exist_ok=True)
    os.makedirs(os.path.join(labels_output, folder), exist_ok=True)

# Helper to convert to YOLO format
def convert_bbox(xmin, ymin, xmax, ymax, img_w, img_h):
    x_center = (xmin + xmax) / 2 / img_w
    y_center = (ymin + ymax) / 2 / img_h
    width = (xmax - xmin) / img_w
    height = (ymax - ymin) / img_h
    return [x_center, y_center, width, height]

# Prepare dataset list
all_files = [f for f in os.listdir(bbox_dir) if f.endswith(".csv")]
random.shuffle(all_files)

# Split into 70% train, 15% val, 15% test
total = len(all_files)
train_split = int(total * 0.7)
val_split = int(total * 0.85)

train_files = all_files[:train_split]
val_files = all_files[train_split:val_split]
test_files = all_files[val_split:]

def process_files(file_list, split):
    for csv_file in file_list:
        image_id = csv_file.replace(".csv", "")
        image_path = os.path.join(images_dir, image_id + ".jpg")
        label_path = os.path.join(labels_output, split, image_id + ".txt")

        if not os.path.exists(image_path):
            continue

        # Load image
        with Image.open(image_path) as img:
            w, h = img.size

        # Read bbox CSV
        df = pd.read_csv(os.path.join(bbox_dir, csv_file))

        # Write YOLO txt label
        with open(label_path, "w") as f:
            for _, row in df.iterrows():
                if row['class_name'].lower() != 'polyp':
                    continue
                xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']
                x_center, y_center, width, height = convert_bbox(xmin, ymin, xmax, ymax, w, h)
                f.write(f"0 {x_center} {y_center} {width} {height}\n")

        # Copy image
        shutil.copy(image_path, os.path.join(images_output, split, image_id + ".jpg"))

# Process each split
process_files(train_files, "train")
process_files(val_files, "val")
process_files(test_files, "test")

# Create data.yaml (Note: test isn't used in YOLOv5 training, only for eval manually)
yaml_text = f"""
path: {os.path.abspath(output_dir)}
train: images/train
val: images/val

nc: 1
names: ['polyp']
"""

with open(os.path.join(output_dir, "data.yaml"), "w") as f:
    f.write(yaml_text.strip())

print("✅ YOLO dataset with train, val, and test prepared successfully!")


✅ YOLO dataset with train, val, and test prepared successfully!


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17496, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 17496 (delta 2), reused 0 (delta 0), pack-reused 17491 (from 3)
Receiving objects: 100% (17496/17496), 16.56 MiB | 10.88 MiB/s, done.
Resolving deltas: 100% (11997/11997), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00

In [ ]:
import os
os.environ['WANDB_MODE'] = 'disabled'
os.environ['WANDB_DISABLED'] = 'true'


In [ ]:
custom_yolo_arch = '''# YOLOv5s Custom Model
nc: 1
depth_multiple: 0.33
width_multiple: 0.50

anchors:
  - [10,13, 16,30, 33,23]
  - [30,61, 62,45, 59,119]
  - [116,90, 156,198, 373,326]

backbone:
  [
    [ -1, 1, Focus, [64, 3] ],
    [ -1, 1, Conv, [128, 3, 2] ],
    [ -1, 3, BottleneckCSP, [128] ],
    [ -1, 1, Conv, [256, 3, 2] ],
    [ -1, 9, BottleneckCSP, [256] ],
    [ -1, 1, Conv, [512, 3, 2] ],
    [ -1, 9, BottleneckCSP, [512] ],
    [ -1, 1, Conv, [1024, 3, 2] ],
    [ -1, 1, SPP, [1024, [5, 9, 13]] ],
    [ -1, 3, BottleneckCSP, [1024, False] ]
  ]

head:
  [
    [ -1, 1, Conv, [512, 1, 1] ],
    [ -1, 1, nn.Upsample, [None, 2, 'nearest'] ],
    [ [ -1, 6 ], 1, Concat, [1] ],
    [ -1, 3, BottleneckCSP, [512] ],

    [ -1, 1, Conv, [256, 1, 1] ],
    [ -1, 1, nn.Upsample, [None, 2, 'nearest'] ],
    [ [ -1, 4 ], 1, Concat, [1] ],
    [ -1, 3, BottleneckCSP, [256] ],

    [ -1, 1, Conv, [256, 3, 2] ],
    [ [ -1, 14 ], 1, Concat, [1] ],
    [ -1, 3, BottleneckCSP, [512] ],

    [ -1, 1, Conv, [512, 3, 2] ],
    [ [ -1, 10 ], 1, Concat, [1] ],
    [ -1, 3, BottleneckCSP, [1024] ],

    [ [17, 20, 23], 1, Detect, [nc, anchors] ]
  ]
'''

with open('/content/yolov5/models/mycustom_yolov5s.yaml', 'w') as f:
    f.write(custom_yolo_arch.strip())


In [ ]:
import os
os.environ['WANDB_MODE'] = 'dryrun'


In [ ]:
import argparse
from yolov5.train import main as yolo_train

class TrainArgs:
    def __init__(self):
        self.imgsz = 640
        self.batch_size = 16
        self.epochs = 1
        self.data = "/content/kvasir_yolo/data.yaml"
        self.cfg = "/content/yolov5/models/mycustom_yolov5s.yaml"
        self.weights = ''  # from scratch
        self.name = "custom_yolov5s"
        self.project = "runs/train"
        self.optimizer = "Adam"
        self.patience = 20
        self.exist_ok = True
        self.save_period = 5
        self.device = '0'
        self.resume = False
        self.evolve = False
        self.nosave = False
        self.hyp = "/content/yolov5/data/hyps/hyp.scratch-low.yaml"
        self.single_cls = False
        self.noval = False
        self.workers = 0      # Try single worker
        self.noplots = True   # Disable label plotting
        self.seed = 0
        self.freeze = [0]
        self.cos_lr = False
        self.sync_bn = False
        self.cache = None
        self.rect = False
        self.image_weights = False
        self.quad = False
        self.noautoanchor = False
        self.label_smoothing = 0.0
        self.multi_scale = False

        # Augmentations off
        self.mosaic = 0
        self.mixup = 0
        self.hsv_h = 0
        self.hsv_s = 0
        self.hsv_v = 0
        self.degrees = 0
        self.translate = 0
        self.scale = 0
        self.shear = 0
        self.flipud = 0
        self.fliplr = 0

        # W&B and logging bypass
        self.nolog = True
        self.bbox_interval = -1
        self.entity = None
        self.save_dir = None

opt = argparse.Namespace(**TrainArgs().__dict__)
print("🚀 Training started!")
yolo_train(opt)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
wandb: WARNING Unable to verify login in offline mode.
train: imgsz=640, batch_size=16, epochs=1, data=/content/kvasir_yolo/data.yaml, cfg=/content/yolov5/models/mycustom_yolov5s.yaml, weights=, name=custom_yolov5s, project=runs/train, optimizer=Adam, patience=20, exist_ok=True, save_period=5, device=0, resume=False, evolve=False, nosave=False, hyp=/content/yolov5/data/hyps/hyp.scratch-low.yaml, single_cls=False, noval=False, workers=0, noplots=True, seed=0, freeze=[0], cos_lr=False, sync_bn=False, cache=None, rect=False, image_weights=False, quad=False, noautoanchor=False, label_smoothing=0.0, multi_scale=False, mosaic=0, mixup=0, hsv_h=0, hsv_s=0, hsv_v=0, degrees=0, translate=0, scale=0, shear=0, flipud=0, fliplr=0, nolog=True, bbox_interval=-1, entity=None, save_dir=None


🚀 Training started!


github: up to date with https://github.com/ultralytics/yolov5 ✅


AssertionError: Invalid CUDA '--device 0' requested, use '--device cpu' or pass valid CUDA device(s)

In [ ]:
!python /content/yolov5/train.py \
  --img 640 \
  --batch 16 \
  --epochs 30 \
  --data /content/kvasir_yolo/data.yaml \
  --cfg /content/yolov5/models/mycustom_yolov5s.yaml \
  --weights '' \
  --name custom_yolov5s \
  --project runs/train \
  --optimizer Adam \
  --hyp /content/yolov5/data/hyps/hyp.scratch-low.yaml \
  --device 0 \
  --exist-ok \
  --freeze 0 \
  --workers 0 \
  --rect \
  --nosave


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-06-26 06:14:07.571191: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750918447.591682    9863 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750918447.597887    9863 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: WARNING Unable to verify login in offline mode.
train: weights=, cfg=/content/yolov5/models/mycustom_yolov5s.yaml, data=/content/kvasir_yolo/data.yaml, hyp=/content/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=640, rect=True, resume=False, nos

In [ ]:
hyp_path = "/content/yolov5/data/hyps/thyp.scratch-med.yaml"

hyp_text = """
lr0: 0.001
lrf: 0.01
momentum: 0.937
weight_decay: 0.0005
warmup_epochs: 3.0
warmup_momentum: 0.8
warmup_bias_lr: 0.1
box: 0.05
cls: 0.3
cls_pw: 1.0
obj: 1.0
obj_pw: 1.0
iou_t: 0.2
anchor_t: 4.0
fl_gamma: 0.0
hsv_h: 0.015
hsv_s: 0.4
hsv_v: 0.3
degrees: 0.0
translate: 0.05
scale: 0.4
shear: 0.0
perspective: 0.0
flipud: 0.0
fliplr: 0.3
mosaic: 1.0
mixup: 0.0
copy_paste: 0.0
"""

with open(hyp_path, "w") as f:
    f.write(hyp_text.strip())

print("✅ Saved custom hyp.scratch-med.yaml")


✅ Saved custom hyp.scratch-med.yaml


Attempt 1


In [ ]:
!python train.py \
  --img 640 \
  --batch 16 \
  --epochs 30 \
  --data /content/kvasir_yolo/data.yaml \
  --cfg /content/yolov5/models/mycustom_yolov5s.yaml \
  --weights yolov5s.pt \
  --name custom_yolov5s_med \
  --project runs/train \
  --optimizer Adam \
  --hyp /content/yolov5/data/hyps/thyp.scratch-med.yaml \
  --device 0 \
  --exist-ok \
  --workers 0


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-06-26 06:38:31.468334: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750919911.488536   16102 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750919911.494787   16102 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: WARNING Unable to verify login in offline mode.
train: weights=yolov5s.pt, cfg=/content/yolov5/models/mycustom_yolov5s.yaml, data=/content/kvasir_yolo/data.yaml, hyp=/content/yolov5/data/hyps/thyp.scratch-med.yaml, epochs=30, batch_size=16, imgsz=640, rect=False, resum

In [ ]:
!python detect.py \
  --weights runs/train/custom_yolov5s_med/weights/best.pt \
  --source /content/kvasir_yolo/images/test \
  --img 640 \
  --conf 0.25 \
  --iou 0.45 \
  --save-txt \
  --save-conf \
  --project runs/test \
  --name yolo_test_results \
  --exist-ok


detect: weights=['runs/train/custom_yolov5s_med/weights/best.pt'], source=/content/kvasir_yolo/images/test, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/test, name=yolo_test_results, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-421-g79c4c31d Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
mycustom_YOLOv5s summary: 182 layers, 7246518 parameters, 0 gradients
image 1/150 /content/kvasir_yolo/images/test/cju15wdt3zla10801odjiw7sy.jpg: 544x640 1 polyp, 29.4ms
image 2/150 /content/kvasir_yolo/images/test/cju16whaj0e7n0855q7b6cjkm.jpg: 576x640 (no detections), 30.2ms
image 3/150 /content/kvasir_yolo/images/test/cju17bz250pgd0799u1hqkj5u.

In [ ]:
test_yaml = """
path: /content/kvasir_yolo
train: images/test
val: images/test

nc: 1
names: ['polyp']
"""

with open("/content/kvasir_yolo/test_data.yaml", "w") as f:
    f.write(test_yaml.strip())


In [ ]:
from yolov5.val import run as yolo_val

results = yolo_val(
    data="/content/kvasir_yolo/test_data.yaml",
    weights="runs/train/custom_yolov5s_med/weights/best.pt",
    imgsz=640,
    conf_thres=0.001,
    iou_thres=0.6,
    device=0,
    half=False,
    save_json=False,
    plots=True,
)


YOLOv5 🚀 v7.0-421-g79c4c31d Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
mycustom_YOLOv5s summary: 182 layers, 7246518 parameters, 0 gradients
val: Scanning /content/kvasir_yolo/labels/test... 150 images, 0 backgrounds, 0 corrupt: 100%|██████████| 150/150 [00:00<00:00, 1452.64it/s]
val: New cache created: /content/kvasir_yolo/labels/test.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 5/5 [00:04<00:00,  1.22it/s]
                   all        150        161      0.846      0.853      0.913       0.64
Speed: 0.3ms pre-process, 6.8ms inference, 2.9ms NMS per image at shape (32, 3, 640, 640)
Results saved to ../runs/val/exp


In [ ]:
metrics, _, _ = results
precision, recall, map50, map50_95, *rest = metrics

print("📈 Test Set Evaluation:")
print(f"✔ Precision     : {precision:.4f}")
print(f"✔ Recall        : {recall:.4f}")
print(f"✔ mAP@0.5       : {map50:.4f}")
print(f"✔ mAP@0.5:0.95  : {map50_95:.4f}")


📈 Test Set Evaluation:
✔ Precision     : 0.8460
✔ Recall        : 0.8533
✔ mAP@0.5       : 0.9127
✔ mAP@0.5:0.95  : 0.6396


Attempt 2

In [ ]:
!python train.py \
  --img 640 \
  --batch 16 \
  --epochs 100 \
  --data /content/kvasir_yolo/data.yaml \
  --cfg models/yolov5m.yaml \
  --weights yolov5m.pt \
  --name improved_yolov5m \
  --project runs/train \
  --optimizer Adam \
  --hyp /content/yolov5/data/hyps/hyp.custom.yaml \
  --device 0 \
  --exist-ok \
  --workers 0


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-06-26 09:36:49.445072: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750930609.473912    2283 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750930609.482309    2283 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: WARNING Unable to verify login in offline mode.
train: weights=yolov5m.pt, cfg=models/yolov5m.yaml, data=/content/kvasir_yolo/data.yaml, hyp=/content/yolov5/data/hyps/hyp.custom.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=F

In [ ]:
                                           # ✅ STEP 1: Disable W&B completely
import os
os.environ['WANDB_MODE'] = 'disabled'

# ✅ STEP 2: Imports
import argparse
import torch
from yolov5.train import main as yolo_train
from yolov5.val import run as yolo_val

# ✅ STEP 3: Define all training arguments (including W&B bypass attributes)
class TrainArgs:
    def __init__(self):
        self.imgsz = 640
        self.batch_size = 16
        self.epochs = 30
        self.data = "/content/kvasir_yolo/data.yaml"
        self.cfg = "/content/yolov5/models/mycustom_yolov5s.yaml"
        self.weights = ''  # from scratch
        self.name = "custom_yolov5s"
        self.project = "runs/train"
        self.optimizer = "Adam"
        self.patience = 20
        self.exist_ok = True
        self.save_period = 5
        self.device = '0'
        self.resume = False
        self.evolve = False
        self.nosave = False
        self.hyp = "/content/yolov5/data/hyps/hyp.scratch-low.yaml"
        self.single_cls = False
        self.noval = False
        self.workers = 0      # Try single worker
        self.noplots = True   # Disable label plotting
        self.seed = 0
        self.freeze = [0]
        self.cos_lr = False
        self.sync_bn = False
        self.cache = None
        self.rect = False
        self.image_weights = False
        self.quad = False
        self.noautoanchor = False
        self.label_smoothing = 0.0
        self.multi_scale = False

        # Augmentations off
        self.mosaic = 0
        self.mixup = 0
        self.hsv_h = 0
        self.hsv_s = 0
        self.hsv_v = 0
        self.degrees = 0
        self.translate = 0
        self.scale = 0
        self.shear = 0
        self.flipud = 0
        self.fliplr = 0

        # W&B and logging bypass
        self.nolog = True
        self.bbox_interval = -1
        self.entity = None
        self.save_dir = None




Attempt 2

In [ ]:
!python train.py \
  --img 640 \
  --batch 16 \
  --epochs 100 \
  --data /content/kvasir_yolo/data.yaml \
  --cfg /content/yolov5/models/mycustom_yolov5s.yaml \
  --weights yolov5s.pt \
  --name custom_yolov5s_med \
  --project runs/train \
  --optimizer Adam \
  --hyp /content/yolov5/data/hyps/thyp.scratch-med.yaml \
  --device 0 \
  --exist-ok \
  --workers 0


Streaming output truncated to the last 5000 lines.
      59/99      5.41G    0.02906    0.01516          0         31        640:  44% 26/59 [00:15<00:18,  1.76it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      59/99      5.41G    0.02897    0.01509          0         31        640:  46% 27/59 [00:15<00:18,  1.73it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      59/99      5.41G    0.02889    0.01499          0         28        640:  47% 28/59 [00:16<00:17,  1.73it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      59/99      5.41G    0.02898    

In [ ]:
!python detect.py \
  --weights runs/train/custom_yolov5s_med/weights/best.pt \
  --source /content/kvasir_yolo/images/test \
  --img 640 \
  --conf 0.25 \
  --iou 0.45 \
  --save-txt \
  --save-conf \
  --project runs/test \
  --name yolo_test_results \
  --exist-ok


detect: weights=['runs/train/custom_yolov5s_med/weights/best.pt'], source=/content/kvasir_yolo/images/test, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/test, name=yolo_test_results, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-421-g79c4c31d Python-3.11.13 torch-2.6.0+cu124 CPU

Traceback (most recent call last):
  File "/content/yolov5/yolov5/yolov5/yolov5/yolov5/yolov5/yolov5/yolov5/detect.py", line 438, in <module>
    main(opt)
  File "/content/yolov5/yolov5/yolov5/yolov5/yolov5/yolov5/yolov5/yolov5/detect.py", line 433, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    

In [ ]:
test_yaml = """
path: /content/kvasir_yolo
train: images/test
val: images/test

nc: 1
names: ['polyp']
"""

with open("/content/kvasir_yolo/test_data.yaml", "w") as f:
    f.write(test_yaml.strip())


In [ ]:
from yolov5.val import run as yolo_val

results = yolo_val(
    data="/content/kvasir_yolo/test_data.yaml",
    weights="runs/train/custom_yolov5s_med/weights/best.pt",
    imgsz=640,
    conf_thres=0.001,
    iou_thres=0.6,
    device=0,
    half=False,
    save_json=False,
    plots=True,
)


YOLOv5 🚀 v7.0-421-g79c4c31d Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
mycustom_YOLOv5s summary: 182 layers, 7246518 parameters, 0 gradients
val: Scanning /content/kvasir_yolo/labels/test... 150 images, 0 backgrounds, 0 corrupt: 100%|██████████| 150/150 [00:00<00:00, 1894.22it/s]
val: New cache created: /content/kvasir_yolo/labels/test.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 5/5 [00:04<00:00,  1.22it/s]
                   all        150        158      0.943      0.949      0.976      0.794
Speed: 0.3ms pre-process, 6.5ms inference, 3.1ms NMS per image at shape (32, 3, 640, 640)
Results saved to runs/val/exp


In [ ]:
# Step 1: Clone YOLOv5 repo
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

# Step 2: Add yolov5 to Python path
import sys
sys.path.append('/content/yolov5')

# Step 3: Now import correctly
from models.common import DetectMultiBackend
from utils.datasets import LoadImages
from utils.general import non_max_suppression, scale_coords
from utils.augmentations import letterbox


Cloning into 'yolov5'...
remote: Enumerating objects: 17496, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 17496 (delta 2), reused 0 (delta 0), pack-reused 17491 (from 3)
Receiving objects: 100% (17496/17496), 16.54 MiB | 15.03 MiB/s, done.
Resolving deltas: 100% (11990/11990), done.
/content/yolov5/yolov5


ModuleNotFoundError: No module named 'utils.datasets'

In [ ]:
# Clone YOLOv5 repo and install dependencies
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

# Add yolov5 folder to path for imports
import sys
import os
sys.path.append(os.getcwd())  # Adds current yolov5 directory

# ✅ Now do your imports — this will work
from models.common import DetectMultiBackend
from utils.datasets import LoadImages
from utils.general import non_max_suppression, scale_coords


Cloning into 'yolov5'...
remote: Enumerating objects: 17496, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 17496 (delta 2), reused 0 (delta 0), pack-reused 17491 (from 3)
Receiving objects: 100% (17496/17496), 16.56 MiB | 15.92 MiB/s, done.
Resolving deltas: 100% (11997/11997), done.
/content/yolov5/yolov5/yolov5/yolov5


ModuleNotFoundError: No module named 'utils.datasets'

In [ ]:
# ============================
# Step 0: Setup
# ============================
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

# Add yolov5 to path
import sys
sys.path.append('/content/yolov5')  # Replace with your path if local

# ============================
# Step 1: Imports
# ============================
import torch
import yaml
import cv2
import os
from pathlib import Path
import matplotlib.pyplot as plt

from models.common import DetectMultiBackend
from utils.datasets import LoadImages
from utils.general import non_max_suppression, scale_coords

# ============================
# Step 2: Configuration
# ============================
weights_path = 'runs/train/custom_yolov5s_med/weights/best.pt'
data_yaml_path = '/content/kvasir_yolo/test_data.yaml'
imgsz = 640
conf_thres = 0.25
iou_thres = 0.45
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Load class names
with open(data_yaml_path) as f:
    data = yaml.safe_load(f)
names = data['names']

# Load model
model = DetectMultiBackend(weights_path, device=device)
stride, pt = model.stride, model.pt
model.eval()

# Load dataset
dataset = LoadImages(data['val'], img_size=imgsz, stride=stride, auto=pt)

# ============================
# Step 3: Inference + Visualization
# ============================
for path, img, im0s, vid_cap, s in dataset:
    im = torch.from_numpy(img).to(device).float()
    im /= 255.0
    im = im.unsqueeze(0)

    # Inference
    pred = model(im)
    pred = non_max_suppression(pred, conf_thres, iou_thres)

    # Read ground truth labels
    label_path = Path(path).with_suffix('.txt').as_posix().replace('images', 'labels')
    gt_boxes = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f.readlines():
                cls, x, y, w, h = map(float, line.strip().split())
                x1 = int((x - w / 2) * im0s.shape[1])
                y1 = int((y - h / 2) * im0s.shape[0])
                x2 = int((x + w / 2) * im0s.shape[1])
                y2 = int((y + h / 2) * im0s.shape[0])
                gt_boxes.append((int(cls), x1, y1, x2, y2))

    # Draw ground truth
    im_gt = im0s.copy()
    for cls, x1, y1, x2, y2 in gt_boxes:
        cv2.rectangle(im_gt, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(im_gt, f"GT: {names[cls]}", (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Draw predictions
    im_pred = im0s.copy()
    if pred[0] is not None and len(pred[0]):
        pred[0][:, :4] = scale_coords(im.shape[2:], pred[0][:, :4], im0s.shape).round()
        for *xyxy, conf, cls in pred[0]:
            x1, y1, x2, y2 = map(int, xyxy)
            cv2.rectangle(im_pred, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(im_pred, f"P: {names[int(cls)]} {conf:.2f}", (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Show side-by-side
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    axs[0].imshow(cv2.cvtColor(im_gt, cv2.COLOR_BGR2RGB))
    axs[0].set_title("Ground Truth")
    axs[0].axis('off')

    axs[1].imshow(cv2.cvtColor(im_pred, cv2.COLOR_BGR2RGB))
    axs[1].set_title("Prediction")
    axs[1].axis('off')

    plt.tight_layout()
    plt.show()


Cloning into 'yolov5'...
remote: Enumerating objects: 17496, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 17496 (delta 2), reused 0 (delta 0), pack-reused 17491 (from 3)
Receiving objects: 100% (17496/17496), 16.56 MiB | 11.86 MiB/s, done.
Resolving deltas: 100% (11997/11997), done.
/content/yolov5/yolov5/yolov5


ModuleNotFoundError: No module named 'utils.datasets'

In [ ]:
metrics, _, _ = results
precision, recall, map50, map50_95, *rest = metrics

print("📈 Test Set Evaluation:")
print(f"✔ Precision     : {precision:.4f}")
print(f"✔ Recall        : {recall:.4f}")
print(f"✔ mAP@0.5       : {map50:.4f}")
print(f"✔ mAP@0.5:0.95  : {map50_95:.4f}")


📈 Test Set Evaluation:
✔ Precision     : 0.9434
✔ Recall        : 0.9486
✔ mAP@0.5       : 0.9758
✔ mAP@0.5:0.95  : 0.7942


In [ ]:
!python val.py \
  --data /content/kvasir_yolo/data.yaml \
  --weights "/content/final_best.pt" \
  --task test \
  --save-txt \
  --save-conf \
  --save-hybrid \
  --project runs/val \
  --name reval_custom \
  --exist-ok


python3: can't open file '/content/val.py': [Errno 2] No such file or directory


In [ ]:
!python detect.py \
  --weights /content/final_best.pt \
  --source /content/kvasir_yolo/images/test \
  --img 640 \
  --conf 0.25 \
  --save-txt \
  --save-conf \
  --project runs/predict \
  --name custom_pred \
  --exist-ok


python3: can't open file '/content/detect.py': [Errno 2] No such file or directory


In [ ]:
import os
!ls -l /content | grep pt


-rw-r--r-- 1 root root 14922052 Jul  1 11:39 final_best (2).pt


In [ ]:
!mv "/content/final_best .pt" "/content/final_best.pt"


mv: cannot stat '/content/final_best .pt': No such file or directory


In [ ]:
!python detect.py \
  --weights /content/final_best.pt \
  --source /content/kvasir_yolo/images/test \
  --img 640 \
  --conf 0.25 \
  --save-txt \
  --save-conf \
  --project runs/predict \
  --name custom_pred \
  --exist-ok


python3: can't open file '/content/detect.py': [Errno 2] No such file or directory
